# Price forecast based on weather, consumption, generation, etc.

Read in data and generate a dataframe for price forecast based on weather, consumption, generation, etc.

Note, that this notebook is just for preparation of input data!!!

In [55]:
import pandas as pd
import numpy as np
import holidays

In [57]:
def mark_holidays_and_weekends(date):
    """
    Returns True if the date is either a German holiday or a weekend
    """
    # Create German holidays object
    german_holidays = holidays.Germany()
    
    # Check if it's a weekend (Saturday=5, Sunday=6)
    is_weekend = date.weekday() >= 5
    
    # Check if it's a holiday
    is_holiday = date in german_holidays
    
    return is_weekend or is_holiday

## prepare dataframe for TFT (TimeSeriesDataSet)

In [56]:
data = pd.read_csv('../data/dataframe.csv',index_col=0, sep=',', decimal='.')
data.index = pd.to_datetime(data.index)
data.sort_index(inplace=True)
data.replace(0., np.nan)
data


,Actual consumption grid load [MWh] Calculated resolutions,Actual consumption Grid load incl. hydro pumped storage [MWh] Calculated resolutions,Actual consumption Hydro pumped storage [MWh] Calculated resolutions,Actual consumption Residual load [MWh] Calculated resolutions,Actual Generation Biomass [MWh] Calculated resolutions,Actual Generation Hydropower [MWh] Calculated resolutions,Actual Generation Wind offshore [MWh] Calculated resolutions,Actual Generation Wind onshore [MWh] Calculated resolutions,Actual Generation Photovoltaics [MWh] Calculated resolutions,Actual Generation Other renewable [MWh] Calculated resolutions,...,Forecasted Generation DA Total [MWh] Original resolutions,Forecasted Generation DA Photovoltaics and wind [MWh] Calculated resolutions,Forecasted Generation DA Wind offshore [MWh] Calculated resolutions,Forecasted Generation DA Wind onshore [MWh] Calculated resolutions,Forecasted Generation DA Photovoltaics [MWh] Calculated resolutions,Forecasted Generation DA Other [MWh] Original resolutions,Generation Forecast ID Photovoltaics and wind [MWh] Calculated resolutions,Generation Forecast ID Wind offshore [MWh] Calculated resolutions,Generation Forecast ID Wind onshore [MWh] Calculated resolutions,Generation Forecast ID Photovoltaics [MWh] Calculated resolutions
Start date,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,43721.75,44994.75,1273.00,20263.75,4755.00,1567.50,3134.00,20324.00,0.0,134.75,...,51071.0,23948.75,5069.00,18879.75,0.00,27122.25,23043.25,3749.25,19294.00,0.00
2019-01-01 01:00:00,42069.00,43479.75,1410.75,16885.00,4753.00,1565.00,2868.25,22315.75,0.0,133.50,...,51084.0,25668.75,5042.25,20626.50,0.00,25415.25,25010.50,3774.75,21235.75,0.00
2019-01-01 02:00:00,40508.00,42984.75,2476.75,14854.25,4710.00,1524.75,2460.25,23193.50,0.0,137.75,...,51513.0,27384.00,5028.50,22355.50,0.00,24129.00,25826.00,3523.25,22302.75,0.00
2019-01-01 03:00:00,39682.50,42967.00,3284.50,12477.25,4685.25,1508.50,2696.50,24508.75,0.0,139.50,...,52693.0,29010.25,4977.75,24032.50,0.00,23682.75,26899.25,3481.75,23417.50,0.00
2019-01-01 04:00:00,39437.50,43153.25,3715.75,10486.25,4696.75,1470.25,2578.00,26373.25,0.0,139.50,...,53666.0,30359.25,4907.25,25452.00,0.00,23306.75,28624.50,3760.75,24863.75,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01 19:00:00,45341.25,45819.25,478.00,14974.50,4204.50,2141.50,6282.75,20698.00,3386.0,93.50,...,51513.0,30641.75,5383.25,22018.00,3240.50,20871.25,29708.50,5457.25,21031.00,3220.25
2024-05-01 20:00:00,45605.25,45623.00,17.75,16802.25,4258.75,2079.25,6396.00,22066.00,341.0,95.50,...,50233.0,29849.00,5312.00,24222.75,314.25,20384.00,28932.00,5435.50,23166.00,330.50
2024-05-01 21:00:00,45397.50,45421.25,23.75,14300.25,4179.75,2070.50,6202.00,24891.75,NaN,96.75,...,51634.0,31458.75,5327.00,26128.25,NaN,20175.25,30749.25,5472.00,25272.25,5.00


In [91]:
data.columns

Index(['Actual consumption grid load [MWh] Calculated resolutions',
       'Actual consumption Grid load incl. hydro pumped storage [MWh] Calculated resolutions',
       'Actual consumption Hydro pumped storage [MWh] Calculated resolutions',
       'Actual consumption Residual load [MWh] Calculated resolutions',
       'Actual Generation Biomass [MWh] Calculated resolutions',
       'Actual Generation Hydropower [MWh] Calculated resolutions',
       'Actual Generation Wind offshore [MWh] Calculated resolutions',
       'Actual Generation Wind onshore [MWh] Calculated resolutions',
       'Actual Generation Photovoltaics [MWh] Calculated resolutions',
       'Actual Generation Other renewable [MWh] Calculated resolutions',
       'Actual Generation Nuclear [MWh] Calculated resolutions',
       'Actual Generation Lignite [MWh] Calculated resolutions',
       'Actual Generation Hard coal [MWh] Calculated resolutions',
       'Actual Generation Fossil gas [MWh] Calculated resolutions',
   

In [84]:
df_list = []
earliest_time = data.index.min()
# Forward fill missing values
data = data.ffill()
# Handle remaining infinite values
data = data.replace([np.inf, -np.inf], np.nan)
data = data.ffill()

ts = data['Dayahead Prices Germany/Luxembourg [€/MWh] Original resolutions']

start_date = min(ts.fillna(method='ffill').dropna().index)
end_date = max(ts.fillna(method='bfill').dropna().index)

tmp = data
date = tmp.index

active_range = (ts.index >= start_date) & (ts.index <= end_date)
ts = ts[active_range].fillna(0.)

tmp['hours_from_start'] = (date - earliest_time).seconds / 60 / 60 + (date - earliest_time).days * 24
tmp['hours_from_start'] = tmp['hours_from_start'].astype('int')

tmp['days_from_start'] = (date - earliest_time).days

tmp['date'] = date
tmp['zone'] = 'Deutschland/Luxemburg [€/MWh]'
tmp['hour'] = date.hour
tmp['day_of_week'] = date.dayofweek
tmp['month'] = date.month
tmp['year'] = date.year
tmp['day'] = date.day
# Apply the function to create the new column
tmp['is_holiday_or_weekend'] = tmp['date'].apply(mark_holidays_and_weekends)
tmp['GWL'] = 0.

df_list.append(tmp)

time_df = pd.concat(df_list).reset_index(drop=True)


C:\Users\Anwender\AppData\Local\Temp\ipykernel_3892\2469855803.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  start_date = min(ts.fillna(method='ffill').dropna().index)
C:\Users\Anwender\AppData\Local\Temp\ipykernel_3892\2469855803.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  end_date = max(ts.fillna(method='bfill').dropna().index)


In [85]:
time_df

,Actual consumption grid load [MWh] Calculated resolutions,Actual consumption Grid load incl. hydro pumped storage [MWh] Calculated resolutions,Actual consumption Hydro pumped storage [MWh] Calculated resolutions,Actual consumption Residual load [MWh] Calculated resolutions,Actual Generation Biomass [MWh] Calculated resolutions,Actual Generation Hydropower [MWh] Calculated resolutions,Actual Generation Wind offshore [MWh] Calculated resolutions,Actual Generation Wind onshore [MWh] Calculated resolutions,Actual Generation Photovoltaics [MWh] Calculated resolutions,Actual Generation Other renewable [MWh] Calculated resolutions,...,days_from_start,date,zone,hour,day_of_week,month,year,day,is_holiday_or_weekend,GWL
0,43721.75,44994.75,1273.00,20263.75,4755.00,1567.50,3134.00,20324.00,0.0,134.75,...,0,2019-01-01 00:00:00,Deutschland/Luxemburg [€/MWh],0,1,1,2019,1,True,0.0
1,42069.00,43479.75,1410.75,16885.00,4753.00,1565.00,2868.25,22315.75,0.0,133.50,...,0,2019-01-01 01:00:00,Deutschland/Luxemburg [€/MWh],1,1,1,2019,1,True,0.0
2,40508.00,42984.75,2476.75,14854.25,4710.00,1524.75,2460.25,23193.50,0.0,137.75,...,0,2019-01-01 02:00:00,Deutschland/Luxemburg [€/MWh],2,1,1,2019,1,True,0.0
3,39682.50,42967.00,3284.50,12477.25,4685.25,1508.50,2696.50,24508.75,0.0,139.50,...,0,2019-01-01 03:00:00,Deutschland/Luxemburg [€/MWh],3,1,1,2019,1,True,0.0
4,39437.50,43153.25,3715.75,10486.25,4696.75,1470.25,2578.00,26373.25,0.0,139.50,...,0,2019-01-01 04:00:00,Deutschland/Luxemburg [€/MWh],4,1,1,2019,1,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47376,45341.25,45819.25,478.00,14974.50,4204.50,2141.50,6282.75,20698.00,3386.0,93.50,...,1947,2024-05-01 19:00:00,Deutschland/Luxemburg [€/MWh],19,2,5,2024,1,True,0.0
47377,45605.25,45623.00,17.75,16802.25,4258.75,2079.25,6396.00,22066.00,341.0,95.50,...,1947,2024-05-01 20:00:00,Deutschland/Luxemburg [€/MWh],20,2,5,2024,1,True,0.0
47378,45397.50,45421.25,23.75,14300.25,4179.75,2070.50,6202.00,24891.75,341.0,96.75,...,1947,2024-05-01 21:00:00,Deutschland/Luxemburg [€/MWh],21,2,5,2024,1,True,0.0
47379,43635.75,44060.00,424.25,13006.75,4066.00,2039.75,5658.50,24967.50,3.0,96.25,...,1947,2024-05-01 22:00:00,Deutschland/Luxemburg [€/MWh],22,2,5,2024,1,True,0.0


In [86]:
GWL = pd.read_excel('../data/GWL.xlsx')
GWL['month'] = GWL['year'].astype('str')
GWL['month'] = GWL['month'].str[:-4]
GWL['year'] = GWL['year'].astype('str')
GWL['year'] = GWL['year'].str[-4:]
GWL

,year,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,month
0,2018,15,5,5,5,4,29,6,10,1,...,10,10,5,24,31,31,28,39,4,10
1,2018,39,19,16,29,28,29,29,19,29,...,34,39,31,12,11,1,9,29,0,11
2,2018,4,39,20,15,5,10,9,20,40,...,20,15,5,10,10,5,5,10,10,12
3,2019,5,15,2,2,10,2,10,40,32,...,33,11,15,10,39,31,14,21,34,1
4,2019,39,39,11,1,5,6,29,9,19,...,2,7,1,2,1,5,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2025,9,35,15,5,29,29,14,19,39,...,19,9,29,19,29,19,39,29,15,1
76,2025,11,5,5,1,5,11,31,29,26,...,19,9,29,16,39,31,0,0,0,2
77,2025,2,1,5,1,1,4,4,33,34,...,39,36,10,10,2,1,6,20,15,3
78,2025,12,3,3,1,5,2,5,2,15,...,19,16,17,1,1,1,6,6,0,4


In [123]:
for ii, row in data.iterrows():
    y = row['year']
    m = row['month']
    d = row['day']
    i = GWL.index[(GWL['year'].astype('int') == y) & (GWL['month'].astype('int') == m)]
    test = GWL[d]
    data.loc[ii,'GWL'] = test[i].item()


In [124]:
data

,Actual consumption grid load [MWh] Calculated resolutions,Actual consumption Grid load incl. hydro pumped storage [MWh] Calculated resolutions,Actual consumption Hydro pumped storage [MWh] Calculated resolutions,Actual consumption Residual load [MWh] Calculated resolutions,Actual Generation Biomass [MWh] Calculated resolutions,Actual Generation Hydropower [MWh] Calculated resolutions,Actual Generation Wind offshore [MWh] Calculated resolutions,Actual Generation Wind onshore [MWh] Calculated resolutions,Actual Generation Photovoltaics [MWh] Calculated resolutions,Actual Generation Other renewable [MWh] Calculated resolutions,...,days_from_start,date,zone,hour,day_of_week,month,year,day,is_holiday_or_weekend,GWL
Start date,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,43721.75,44994.75,1273.00,20263.75,4755.00,1567.50,3134.00,20324.00,0.0,134.75,...,0,2019-01-01 00:00:00,Deutschland/Luxemburg [€/MWh],0,1,1,2019,1,True,5.0
2019-01-01 01:00:00,42069.00,43479.75,1410.75,16885.00,4753.00,1565.00,2868.25,22315.75,0.0,133.50,...,0,2019-01-01 01:00:00,Deutschland/Luxemburg [€/MWh],1,1,1,2019,1,True,5.0
2019-01-01 02:00:00,40508.00,42984.75,2476.75,14854.25,4710.00,1524.75,2460.25,23193.50,0.0,137.75,...,0,2019-01-01 02:00:00,Deutschland/Luxemburg [€/MWh],2,1,1,2019,1,True,5.0
2019-01-01 03:00:00,39682.50,42967.00,3284.50,12477.25,4685.25,1508.50,2696.50,24508.75,0.0,139.50,...,0,2019-01-01 03:00:00,Deutschland/Luxemburg [€/MWh],3,1,1,2019,1,True,5.0
2019-01-01 04:00:00,39437.50,43153.25,3715.75,10486.25,4696.75,1470.25,2578.00,26373.25,0.0,139.50,...,0,2019-01-01 04:00:00,Deutschland/Luxemburg [€/MWh],4,1,1,2019,1,True,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01 19:00:00,45341.25,45819.25,478.00,14974.50,4204.50,2141.50,6282.75,20698.00,3386.0,93.50,...,1947,2024-05-01 19:00:00,Deutschland/Luxemburg [€/MWh],19,2,5,2024,1,True,28.0
2024-05-01 20:00:00,45605.25,45623.00,17.75,16802.25,4258.75,2079.25,6396.00,22066.00,341.0,95.50,...,1947,2024-05-01 20:00:00,Deutschland/Luxemburg [€/MWh],20,2,5,2024,1,True,28.0
2024-05-01 21:00:00,45397.50,45421.25,23.75,14300.25,4179.75,2070.50,6202.00,24891.75,341.0,96.75,...,1947,2024-05-01 21:00:00,Deutschland/Luxemburg [€/MWh],21,2,5,2024,1,True,28.0


In [131]:
data.to_csv('../data/dataframe.csv')